In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy
import numpy as np
import pandas as pd
import itertools
import os
import csv
import re
import nltk
from nltk.tokenize import sent_tokenize

# Import adjustText, initialize list of texts
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import PCA



from sklearn import metrics
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score,fbeta_score

from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import svm


import seaborn as sns
import matplotlib.pyplot as plt



from IPython.display import SVG


#Seed Random Numbers with the TensorFlow Backend
from numpy.random import seed
seed(1)

In [ ]:
df_train = pd.read_excel('Training Dataset.xlsx')
df_train.shape, df_train.columns 

((3564, 3), Index(['Tweets', 'label', 'S/G'], dtype='object'))

In [ ]:
df_train.drop('S/G', inplace=True, axis=1)

In [ ]:
df_train.head()

,Tweets,label
0,بکواس مت کرو,1
1,تمہاری ہیجڑا فورس ایک نہتے کے سامنے بکری بنی ہ...,1
2,آفیسر سمیتبھارتی فوجی جہنم واصل،بنکرز تباہ بھا...,1
3,غدار منافق میر اللہ تمہیں زلیل کرے,1
4,اگست پر آپ بھارت کو کیا پیغام دینا چاہیں گے؟ م...,1


In [ ]:
print("Total Training Dataset:" + str(len(df_train)))
print("yes class:" + str(len(df_train[(df_train['label'] == 1)])))
print("no class:" + str(len(df_train[(df_train['label'] == 0)])))

Total Training Dataset:3564
yes class:1782
no class:1782


In [ ]:
df_test = pd.read_excel('data_test.xlsx', usecols=['Tweet', 'label'])
df_test.head()

,Tweet,label
0,وزیراعل ی پنجاب کا ضلع راولپنڈی کا دورہ لینڈسل...,0
1,الحمدلله پورے پاکستان کے وکلاء برادری کا کہنا ...,1
2,جنوری مہلت ختم الٹی گنتی شروع ن لیگ کے خلا...,1
3,کروناکی آڑ میں حکومت اورفوج جوکھیل قادیانیوں ک...,1
4,حکومت ایکشن میں آئے ورنہ اگر ہم ایکشن میں آگئے...,1


In [ ]:
df_full = pd.concat([df_train, df_test])

In [ ]:
threat_df = df_full[(df_full['label'] == 1)]
threat_df['word_count'] = threat_df['Tweets'].apply(lambda x: len(str(x).split()))
threat_df['char_count'] = threat_df['Tweets'].str.len() ## this also includes spaces


df_threat_stat = threat_df[['word_count', 'char_count']].agg(['sum','mean'])

print("=========== Threat Dataset ===========")
print(threat_df.shape)
print(df_threat_stat)
print("=========== Threat Dataset ===========")


=========== Threat Dataset ===========
(2089, 5)
        word_count     char_count
sum   30825.000000  134408.000000
mean     14.755864      75.425365
=========== Threat Dataset ===========


In [ ]:
non_threat_df = df_full[(df_full['label'] == 0)]
non_threat_df['word_count'] = non_threat_df['Tweets'].apply(lambda x: len(str(x).split()))
non_threat_df['char_count'] = non_threat_df['Tweets'].str.len() ## this also includes spaces


df_non_threat_stat = non_threat_df[['word_count', 'char_count']].agg(['sum','mean'])

print("=========== Dataset non-threat ===========")
print(non_threat_df.shape)
print(df_non_threat_stat)
print("=========== Dataset non-threat ===========")

=========== Dataset non-threat ===========
(2410, 5)
        word_count     char_count
sum   31763.000000  140225.000000
mean     13.179668      78.689675
=========== Dataset non-threat ===========


## Baseline Model

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
def extract_ngrams(data, num=1):
    n_grams = nltk.ngrams(nltk.word_tokenize(data), num)
    return [ ' '.join(grams) for grams in n_grams]

In [ ]:
df_train['unigrams'] = df_train['Tweets'].apply(extract_ngrams)
df_train['bigrams'] = df_train['Tweets'].apply(lambda x: extract_ngrams(x, 2))

df_test['unigrams'] = df_test['Tweet'].apply(extract_ngrams)
df_test['bigrams'] = df_test['Tweet'].apply(lambda x: extract_ngrams(x, 2))

## Vocabulary Set-Up

In [ ]:
from collections import Counter

In [ ]:
uni = Counter()
for idx, row in df_train.iterrows(): 
    uni.update(row['unigrams'])

In [ ]:
df_vocab = pd.DataFrame.from_dict(uni, orient='index', columns=['count'])
df_vocab.reset_index(inplace=True)
df_vocab.columns = ['uniram', 'count']

In [ ]:
df_vocab.describe()

,count
count,8923.000000
mean,6.913482
std,44.381307
min,1.000000
25%,1.000000
50%,1.000000
75%,3.000000
max,1629.000000


In [ ]:
df_vocab['count'].sum()

61689

In [ ]:
df_vocab.shape, df_vocab[df_vocab['count'] >= 3].shape

((8923, 2), (2482, 2))

In [ ]:
vocab_uni = set(df_vocab[df_vocab['count'] >= 3]['uniram'])
len(vocab_uni)

2482

In [ ]:
bi = Counter()
for idx, row in df_train.iterrows(): 
    bi.update(row['bigrams'])    
len(bi)   

37450

In [ ]:
vocab_bi_df = pd.DataFrame.from_dict(bi, orient='index', columns=['count'])
vocab_bi_df.reset_index(inplace=True)
vocab_bi_df.columns = ['bigram', 'count']

In [ ]:
vocab_bi_df.describe(percentiles=[.25, .5, .75, .8, .85, .9, .95, .99])

,count
count,37450.000000
mean,1.552069
std,2.525382
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
80%,1.000000
85%,2.000000
90%,2.000000


In [ ]:
vocab_bi_df.shape, vocab_bi_df[vocab_bi_df['count'] >= 4].shape

((37450, 2), (1944, 2))

In [ ]:
vocab_bi = set(vocab_bi_df[vocab_bi_df['count'] >= 4]['bigram'])
len(vocab_bi)

1944

In [ ]:
vocab = vocab_uni.union(vocab_bi)
len(vocab)

4426

## Feature Engineering

In [ ]:
import numpy as np

In [ ]:
vocab_uni_li = list(vocab_uni)
vocab_bi_li = list(vocab_bi)

FEATURES = vocab_uni_li+vocab_bi_li + ['OOV_uni', 'OOV_bi']
len(vocab_uni_li) + len(vocab_bi_li), len(FEATURES)


(4426, 4428)

In [ ]:
df_train.shape

(3564, 4)

In [ ]:
stuffing = np.zeros((3564, 4428))
stuffing_df = pd.DataFrame(stuffing, columns = FEATURES)

In [ ]:
df_train.head()

,Tweets,label,unigrams,bigrams
0,بکواس مت کرو,1,"[بکواس, مت, کرو]","[بکواس مت, مت کرو]"
1,تمہاری ہیجڑا فورس ایک نہتے کے سامنے بکری بنی ہ...,1,"[تمہاری, ہیجڑا, فورس, ایک, نہتے, کے, سامنے, بک...","[تمہاری ہیجڑا, ہیجڑا فورس, فورس ایک, ایک نہتے,..."
2,آفیسر سمیتبھارتی فوجی جہنم واصل،بنکرز تباہ بھا...,1,"[آفیسر, سمیتبھارتی, فوجی, جہنم, واصل،بنکرز, تب...","[آفیسر سمیتبھارتی, سمیتبھارتی فوجی, فوجی جہنم,..."
3,غدار منافق میر اللہ تمہیں زلیل کرے,1,"[غدار, منافق, میر, اللہ, تمہیں, زلیل, کرے]","[غدار منافق, منافق میر, میر اللہ, اللہ تمہیں, ..."
4,اگست پر آپ بھارت کو کیا پیغام دینا چاہیں گے؟ م...,1,"[اگست, پر, آپ, بھارت, کو, کیا, پیغام, دینا, چا...","[اگست پر, پر آپ, آپ بھارت, بھارت کو, کو کیا, ک..."


In [ ]:
train_df = pd.concat([df_train, stuffing_df], axis=1)
train_df.shape

(3564, 4432)

In [ ]:
def count_ngram_feats(df):    
    global vocab
    
    total_oov_uni = 0
    total_oov_bi = 0
    
    for idx, row in df.iterrows(): 
        # unigram part 
        ct = Counter()
        ct.update(row['unigrams'])
        oov_ct = 0 
        
        for k in ct.elements(): 
            if k in vocab:
                df[k].iloc[idx] = ct.get(k)
            else:     
                oov_ct += ct.get(k)
        df['OOV_uni'].iloc[idx] = oov_ct
        total_oov_uni += oov_ct

        # bigram part 
        ct = Counter()
        ct.update(row['bigrams'])
        oov_ct = 0 
        
        for k in ct.elements(): 
            if k in vocab:
                df[k].iloc[idx] = ct.get(k)
            else:     
                oov_ct += ct.get(k)
        df['OOV_bi'].iloc[idx] = oov_ct 
        total_oov_bi += oov_ct

    print (total_oov_uni, total_oov_bi)

In [ ]:
count_ngram_feats(train_df)

7767 41719


In [ ]:
train_df[list(vocab)[:3]].sum()

کر لیا    9.0
قید       6.0
اینڈ      4.0
dtype: float64

In [ ]:
train_df[['OOV_uni', 'OOV_bi']].sum()

OOV_uni     7767.0
OOV_bi     41719.0
dtype: float64

## Test Part

In [ ]:
df_test.shape

(935, 4)

In [ ]:
stuffing = np.zeros((935, 4428))
stuffing_df = pd.DataFrame(stuffing, columns=FEATURES)

In [ ]:
test_df = pd.concat([df_test, stuffing_df], axis=1)
test_df.shape

(935, 4432)

In [ ]:
count_ngram_feats(test_df)

3776 14179


## Model

In [ ]:
!pip freeze

absl-py==1.3.0
aeppl==0.0.33
aesara==2.7.9
aiohttp==3.8.3
aiosignal==1.3.1
alabaster==0.7.12
albumentations==1.2.1
altair==4.2.0
appdirs==1.4.4
arviz==0.12.1
astor==0.8.1
astropy==4.3.1
astunparse==1.6.3
async-timeout==4.0.2
asynctest==0.13.0
atari-py==0.2.9
atomicwrites==1.4.1
attrs==22.1.0
audioread==3.0.0
autograd==1.5
Babel==2.11.0
backcall==0.2.0
beautifulsoup4==4.6.3
bleach==5.0.1
blis==0.7.9
bokeh==2.3.3
branca==0.6.0
bs4==0.0.1
CacheControl==0.12.11
cached-property==1.5.2
cachetools==5.2.0
catalogue==2.0.8
certifi==2022.9.24
cffi==1.15.1
cftime==1.6.2
chardet==3.0.4
charset-normalizer==2.1.1
click==7.1.2
clikit==0.6.2
cloudpickle==1.5.0
cmake==3.22.6
cmdstanpy==1.0.8
colorcet==3.0.1
colorlover==0.3.0
community==1.0.0b1
confection==0.0.3
cons==0.4.5
contextlib2==0.5.5
convertdate==2.4.0
crashtest==0.3.1
crcmod==1.7
cufflinks==0.17.3
cvxopt==1.3.0
cvxpy==1.2.2
cycler==0.11.0
cymem==2.0.7
Cython==0.29.32
daft==0.0.4
dask==2022.2.0
datascience==0.17.5
db-dtypes==1.0.4
debugpy==1.0.

In [ ]:
from sklearn.linear_model import LogisticRegression
#from sklearn.ensemble import GradientBoostingClassifier 
#from xgboost import XGBClassifier
from sklearn.model_selection import cross_validate
from sklearn.metrics import classification_report, f1_score, roc_auc_score, accuracy_score
#from sklearn.model_selection import GridSearchCV

## Logistic Regression

In [ ]:
logreg = LogisticRegression(max_iter=1000, class_weight='balanced')

In [ ]:
logreg.fit(train_df[FEATURES], train_df['label'])

LogisticRegression(class_weight='balanced', max_iter=1000)

In [ ]:
y = train_df['label']
y.shape

(3564,)

In [ ]:
y_pred = logreg.predict(train_df[FEATURES])
y_pred_score = logreg.predict_proba(train_df[FEATURES])

In [ ]:
f1_score(y, y_pred), roc_auc_score(y, y_pred_score[:, 1]), accuracy_score(y, y_pred)

(0.9691011235955055, 0.9959956215100247, 0.9691358024691358)

In [ ]:
y_pred_test = logreg.predict(test_df[FEATURES])
y_pred_score_test = logreg.predict_proba(test_df[FEATURES])
y_test = test_df['label']
f1_score(y_test, y_pred_test), roc_auc_score(y_test, y_pred_score_test[:, 1]), accuracy_score(y_test, y_pred_test)

(0.49032258064516127, 0.681489761198365, 0.6620320855614973)

## SVM

In [ ]:
svc = svm.SVC(probability=True, class_weight='balanced')
svc.fit(train_df[FEATURES], train_df['label'])

SVC(class_weight='balanced', probability=True)

In [ ]:
y_pred = svc.predict(train_df[FEATURES])
y_pred_score = svc.predict_proba(train_df[FEATURES])

In [ ]:
f1_score(y, y_pred), roc_auc_score(y, y_pred_score[:, 1]), accuracy_score(y, y_pred)

(0.7542857142857141, 0.8431036893438688, 0.7586980920314254)

In [ ]:
y_pred_test = svc.predict(test_df[FEATURES])
y_pred_score_test = svc.predict_proba(test_df[FEATURES])
y_test = test_df['label']
f1_score(y_test, y_pred_test), roc_auc_score(y_test, y_pred_score_test[:, 1]), accuracy_score(y_test, y_pred_test)

(0.47755834829443455, 0.6731597128571132, 0.6887700534759358)

## Random ForestClassifier

In [ ]:
rf = RandomForestClassifier(max_features=None)
rf.fit(train_df[FEATURES], train_df['label'])

RandomForestClassifier(max_features=None)

In [ ]:
y_pred = rf.predict(train_df[FEATURES])
y_pred_score = rf.predict_proba(train_df[FEATURES])

In [ ]:
f1_score(y, y_pred), roc_auc_score(y, y_pred_score[:, 1]), accuracy_score(y, y_pred)

(0.999438832772166, 0.9999971658220816, 0.999438832772166)

In [ ]:
y_pred_test_rf = rf.predict(test_df[FEATURES])
y_pred_score_test_rf = rf.predict_proba(test_df[FEATURES])
y_test_rf = test_df['label']
f1_score(y_test_rf , y_pred_test_rf), roc_auc_score(y_test_rf, y_pred_score_test_rf[:, 1]), accuracy_score(y_test_rf, y_pred_test_rf)

(0.5117845117845118, 0.6760228427975683, 0.6898395721925134)

## Gradient Boosting Classifier

In [ ]:
gb = GradientBoostingClassifier()
gb.fit(train_df[FEATURES], train_df['label'])

GradientBoostingClassifier()

In [ ]:
y_pred_gb = gb.predict(train_df[FEATURES])
y_pred_score_gb = gb.predict_proba(train_df[FEATURES])

In [ ]:
f1_score(y, y_pred_gb), roc_auc_score(y, y_pred_score_gb[:, 1]), accuracy_score(y, y_pred_gb)

(0.758428703990102, 0.8621232275366206, 0.7808641975308642)

In [ ]:
y_pred_test_gb  = gb.predict(test_df[FEATURES])
y_pred_score_test_gb = gb.predict_proba(test_df[FEATURES])
y_test_gb = test_df['label']
f1_score(y_test_gb , y_pred_test_gb), roc_auc_score(y_test_gb, y_pred_score_test_gb[:, 1]), accuracy_score(y_test_gb, y_pred_test_gb)

(0.5026178010471204, 0.6447514471254591, 0.6951871657754011)

## AdaBoost Classifier

In [ ]:
ad = AdaBoostClassifier()
ad.fit(train_df[FEATURES], train_df['label'])

AdaBoostClassifier()

In [ ]:
y_pred_ad = ad.predict(train_df[FEATURES])
y_pred_score_ad = ad.predict_proba(train_df[FEATURES])

In [ ]:
f1_score(y, y_pred_ad), roc_auc_score(y, y_pred_score_ad[:, 1]), accuracy_score(y, y_pred_ad)

(0.6953919808497906, 0.7934449558561044, 0.7143658810325477)

In [ ]:
y_pred_test_ad  = ad.predict(test_df[FEATURES])
y_pred_score_test_ad = ad.predict_proba(test_df[FEATURES])
y_test_ad = test_df['label']
f1_score(y_test_ad , y_pred_test_ad), roc_auc_score(y_test_ad, y_pred_score_test_ad[:, 1]), accuracy_score(y_test_ad, y_pred_test_ad)

(0.4857142857142857, 0.6369763895516505, 0.6919786096256685)

## GaussianNB

In [ ]:
nb = GaussianNB()
nb.fit(train_df[FEATURES], train_df['label'])

GaussianNB()

In [ ]:
y_pred_nb = nb.predict(train_df[FEATURES])
y_pred_score_nb = nb.predict_proba(train_df[FEATURES])

In [ ]:
f1_score(y, y_pred_nb), roc_auc_score(y, y_pred_score_nb[:, 1]), accuracy_score(y, y_pred_nb)

(0.8792079207920793, 0.9128246235896815, 0.8801907968574635)

In [ ]:
y_pred_test_nb  = nb.predict(test_df[FEATURES])
y_pred_score_test_nb = nb.predict_proba(test_df[FEATURES])
y_test_nb = test_df['label']
f1_score(y_test_nb , y_pred_test_nb), roc_auc_score(y_test_nb, y_pred_score_test_nb[:, 1]), accuracy_score(y_test_nb, y_pred_test_nb)

(0.49779086892488944, 0.6180548351625552, 0.6352941176470588)

## Decision Tree Classifier

In [ ]:
dt = DecisionTreeClassifier(random_state=0)
dt.fit(train_df[FEATURES], train_df['label'])

DecisionTreeClassifier(random_state=0)

In [ ]:
y_pred_dt = dt.predict(train_df[FEATURES])
y_pred_score_dt = dt.predict_proba(train_df[FEATURES])

In [ ]:
f1_score(y, y_pred_dt), roc_auc_score(y, y_pred_score_dt[:, 1]), accuracy_score(y, y_pred_dt)

(0.9994385176866928, 0.9999993701826848, 0.999438832772166)

In [ ]:
y_pred_test_dt  = dt.predict(test_df[FEATURES])
y_pred_score_test_dt = dt.predict_proba(test_df[FEATURES])
y_test_dt = test_df['label']
f1_score(y_test_dt , y_pred_test_dt), roc_auc_score(y_test_dt, y_pred_score_test_dt[:, 1]), accuracy_score(y_test_dt, y_pred_test_dt)

(0.4912790697674419, 0.6064544907570697, 0.6256684491978609)